# PdCrO2 magnetic moments
(model 4 from "H. Takatsu, et al., Phys. Rev. B 89 (2014) 104408.")

In [2]:
import numpy as np
import pandas as pd

# === 参数定义 ===
deg = np.pi / 180  # 角度转弧度
S = 1.0  # 自旋大小（单位向量）

# α_n 和 φ_n （每层）单位为度，从 Figure 5(d)
alpha_deg = [31, 44, 31, 44, 31, 44]
phi_deg = [17, 16, 17, 16, 17, 16]
gamma_deg = [0, 0, 0, 0, 0, 0]  # γ_n = 0 表示自旋平面垂直 z 轴

# ξ_n = +1 (偶数层)，-1 (奇数层)
xi_n = [1 if n % 2 == 0 else -1 for n in range(6)]

# === 单位矢量定义 ===
x_hat = np.array([1, 0, 0])
y_hat = np.array([0, 1, 0])
z_hat = np.array([0, 0, 1])

# α_n 平面方向矢量
def e_alpha(alpha_rad):
    return np.cos(alpha_rad) * x_hat + np.sin(alpha_rad) * y_hat

def e_alpha_prime(alpha_rad):
    return np.cos(alpha_rad + np.pi/2) * x_hat + np.sin(alpha_rad + np.pi/2) * y_hat

# === 计算每层 A, B, C 子晶格的磁矩 ===
magnetic_moments = []

for n in range(6):
    alpha_n = alpha_deg[n] * deg
    phi_n = phi_deg[n] * deg
    gamma_n = gamma_deg[n] * deg
    xi = xi_n[n]

    # 方向矢量
    e_a = e_alpha(alpha_n)
    e_a_p = e_alpha_prime(alpha_n)

    # A_n 自旋方向
    S_A = (np.cos(gamma_n) * np.cos(phi_n) * z_hat +
           np.sin(phi_n) * e_a +
           np.sin(gamma_n) * np.cos(phi_n) * e_a_p)

    # B_n 自旋方向：φ + ξ * 120°
    phi_B = phi_n + xi * 2 * np.pi / 3
    S_B = (np.cos(gamma_n) * np.cos(phi_B) * z_hat +
           np.sin(phi_B) * e_a +
           np.sin(gamma_n) * np.cos(phi_B) * e_a_p)

    # C_n 自旋方向：φ - ξ * 120°
    phi_C = phi_n - xi * 2 * np.pi / 3
    S_C = (np.cos(gamma_n) * np.cos(phi_C) * z_hat +
           np.sin(phi_C) * e_a +
           np.sin(gamma_n) * np.cos(phi_C) * e_a_p)

    # 加入列表
    magnetic_moments.append((f"A{n}", *S_A))
    magnetic_moments.append((f"B{n}", *S_B))
    magnetic_moments.append((f"C{n}", *S_C))

# === 构造 DataFrame 展示结果 ===
df_moments = pd.DataFrame(magnetic_moments, columns=["Site", "Mx", "My", "Mz"])
df_moments


,Site,Mx,My,Mz
0,A0,0.250611,0.150583,0.956305
1,B0,0.584587,0.351255,-0.731354
2,C0,-0.835198,-0.501838,-0.224951
3,A1,0.198277,0.191474,0.961262
4,B1,-0.697972,-0.674024,-0.241922
5,C1,0.499695,0.482550,-0.719340
6,A2,0.250611,0.150583,0.956305
7,B2,0.584587,0.351255,-0.731354
8,C2,-0.835198,-0.501838,-0.224951
9,A3,0.198277,0.191474,0.961262


In [3]:
# === 输出磁矩数据到txt文件 ===
# 保存为txt文件
output_filename = "magnetic_moments.txt"

with open(output_filename, 'w', encoding='utf-8') as f:
    f.write("PdCrO2 磁矩计算结果\n")
    f.write("=" * 50 + "\n")
    f.write("参数设置:\n")
    f.write(f"α_n (度): {alpha_deg}\n")
    f.write(f"φ_n (度): {phi_deg}\n")
    f.write(f"γ_n (度): {gamma_deg}\n")
    f.write(f"ξ_n: {xi_n}\n")
    f.write("\n")
    
    f.write("各原子位点的磁矩方向 (单位向量):\n")
    f.write("-" * 50 + "\n")
    f.write(f"{'Site':<8} {'Mx':<12} {'My':<12} {'Mz':<12}\n")
    f.write("-" * 50 + "\n")
    
    for _, row in df_moments.iterrows():
        f.write(f"{row['Site']:<8} {row['Mx']:<12.6f} {row['My']:<12.6f} {row['Mz']:<12.6f}\n")
    
    f.write("\n")
    f.write("数据说明:\n")
    f.write("- Mx, My, Mz 为磁矩在 x, y, z 方向的分量\n")
    f.write("- 所有分量都已归一化为单位向量\n")
    f.write("- A, B, C 表示三角晶格中的三个子晶格\n")
    f.write("- 数字表示层数 (0-5)\n")

print(f"磁矩数据已保存到文件: {output_filename}")

# 同时输出VASP格式的MAGMOM文件
magmom_filename = "MAGMOM.txt"
with open(magmom_filename, 'w') as f:
    for _, row in df_moments.iterrows():
        f.write(f"{row['Mx']:.6f} {row['My']:.6f} {row['Mz']:.6f}\n")

print(f"VASP格式磁矩数据已保存到文件: {magmom_filename}")

# 显示生成的文件内容预览
print("\n=== 生成的文件内容预览 ===")
print(f"\n{output_filename}:")
with open(output_filename, 'r', encoding='utf-8') as f:
    print(f.read())

print(f"\n{magmom_filename} (前10行):")
with open(magmom_filename, 'r') as f:
    for i, line in enumerate(f):
        if i < 10:
            print(line.strip())
        else:
            break

磁矩数据已保存到文件: magnetic_moments.txt
VASP格式磁矩数据已保存到文件: MAGMOM.txt

=== 生成的文件内容预览 ===

magnetic_moments.txt:
PdCrO2 磁矩计算结果
参数设置:
α_n (度): [31, 44, 31, 44, 31, 44]
φ_n (度): [17, 16, 17, 16, 17, 16]
γ_n (度): [0, 0, 0, 0, 0, 0]
ξ_n: [1, -1, 1, -1, 1, -1]

各原子位点的磁矩方向 (单位向量):
--------------------------------------------------
Site     Mx           My           Mz          
--------------------------------------------------
A0       0.250611     0.150583     0.956305    
B0       0.584587     0.351255     -0.731354   
C0       -0.835198    -0.501838    -0.224951   
A1       0.198277     0.191474     0.961262    
B1       -0.697972    -0.674024    -0.241922   
C1       0.499695     0.482550     -0.719340   
A2       0.250611     0.150583     0.956305    
B2       0.584587     0.351255     -0.731354   
C2       -0.835198    -0.501838    -0.224951   
A3       0.198277     0.191474     0.961262    
B3       -0.697972    -0.674024    -0.241922   
C3       0.499695     0.482550     -0.719340   
A4     

# PdCrO2 POSCAR

1. 首先在cod下载pdcro2.cif文件

sm_isp_SD1817668-standardized_unitcell
1.0
        2.9279999733         0.0000000000         0.0000000000
       -1.4639999866         2.5357223592         0.0000000000
        0.0000000000         0.0000000000        18.1217002869
    O   Cr   Pd
    6    3    3
Cartesian
     0.000000000         0.000000000         1.927605260
     0.000000000         0.000000000        16.194095027
     1.463999987         0.845240786         7.968172022
     1.463999987         0.845240786         4.112961503
     0.000000000         1.690481573        14.008738784
     0.000000000         1.690481573        10.153528265
     0.000000000         0.000000000         9.060850143
     1.463999987         0.845240786        15.101416906
     0.000000000         1.690481573         3.020283381
     0.000000000         0.000000000         0.000000000
     1.463999987         0.845240786         6.040566762
     0.000000000         1.690481573        12.081133525

2. 借助vaaspkit对pdcro2进行扩胞（根号3*根号3）

   vaspkit选项：400 -- 2 1 0 -- 1 2 0 -- 0 0 2

得到:
Generated by VASPKIT code
 1.000000
    4.3919999599999997    2.5357223592000002    0.0000000000000000
    0.0000000001000000    5.0714447184000004    0.0000000000000000
    0.0000000000000000    0.0000000000000000   36.2434005738000025
   O    Cr   Pd
    36    18    18
Cartesian
    1.4639999867000000    2.5357223592000007    1.9276052599999991     O001
    1.4639999867000000    2.5357223592000007   20.0493055469000012     O002
    0.0000000000000000    0.0000000000000000    1.9276052599999991     O003
    0.0000000000000000    0.0000000000000000   20.0493055469000012     O004
    2.9279999734000000    5.0714447184000004    1.9276052599999991     O005
    2.9279999734000000    5.0714447184000004   20.0493055469000012     O006
    1.4639999867000000    2.5357223592000007   16.1940950269999995     O007
    1.4639999867000000    2.5357223592000007   34.3157953138999972     O008
    0.0000000000000000    0.0000000000000000   16.1940950269999995     O009
    0.0000000000000000    0.0000000000000000   34.3157953138999972     O010
    2.9279999734000000    5.0714447184000004   16.1940950269999995     O011
    2.9279999734000000    5.0714447184000004   34.3157953138999972     O012
    2.9279999737000000    3.3809631452000004    7.9681720219999974     O013
    2.9279999737000000    3.3809631452000004   26.0898723088999986     O014
    1.4639999871000002    5.9166855044000002    7.9681720219999974     O015
    1.4639999871000002    5.9166855044000002   26.0898723088999986     O016
    0.0000000004000002    3.3809631452000000    7.9681720219999974     O017
    0.0000000004000002    3.3809631452000000   26.0898723088999986     O018
    2.9279999737000000    3.3809631452000004    4.1129615030000011     O019
    2.9279999737000000    3.3809631452000004   22.2346617899000023     O020
    1.4639999871000002    5.9166855044000002    4.1129615030000011     O021
    1.4639999871000002    5.9166855044000002   22.2346617899000023     O022
    0.0000000004000002    3.3809631452000000    4.1129615030000011     O023
    0.0000000004000002    3.3809631452000000   22.2346617899000023     O024
    1.4639999867000000    4.2262039322000007   14.0087387840000002     O025
    1.4639999867000000    4.2262039322000007   32.1304390709000032     O026
    4.3919999599999997    4.2262039322000007   14.0087387840000002     O027
    4.3919999599999997    4.2262039322000007   32.1304390709000032     O028
    2.9279999732999995    1.6904815729999996   14.0087387840000002     O029
    2.9279999732999995    1.6904815729999996   32.1304390709000032     O030
    1.4639999867000000    4.2262039322000007   10.1535282650000003     O031
    1.4639999867000000    4.2262039322000007   28.2752285518999997     O032
    4.3919999599999997    4.2262039322000007   10.1535282650000003     O033
    4.3919999599999997    4.2262039322000007   28.2752285518999997     O034
    2.9279999732999995    1.6904815729999996   10.1535282650000003     O035
    2.9279999732999995    1.6904815729999996   28.2752285518999997     O036
    1.4639999867000000    2.5357223592000007    9.0608501429999997    Cr001
    1.4639999867000000    2.5357223592000007   27.1825504299000009    Cr002
    0.0000000000000000    0.0000000000000000    9.0608501429999997    Cr003
    0.0000000000000000    0.0000000000000000   27.1825504299000009    Cr004
    2.9279999734000000    5.0714447184000004    9.0608501429999997    Cr005
    2.9279999734000000    5.0714447184000004   27.1825504299000009    Cr006
    2.9279999737000000    3.3809631452000004   15.1014169060000008    Cr007
    2.9279999737000000    3.3809631452000004   33.2231171928999984    Cr008
    1.4639999871000002    5.9166855044000002   15.1014169060000008    Cr009
    1.4639999871000002    5.9166855044000002   33.2231171928999984    Cr010
    0.0000000004000002    3.3809631452000000   15.1014169060000008    Cr011
    0.0000000004000002    3.3809631452000000   33.2231171928999984    Cr012
    1.4639999867000000    4.2262039322000007    3.0202833810000014    Cr013
    1.4639999867000000    4.2262039322000007   21.1419836678999999    Cr014
    4.3919999599999997    4.2262039322000007    3.0202833810000014    Cr015
    4.3919999599999997    4.2262039322000007   21.1419836678999999    Cr016
    2.9279999732999995    1.6904815729999996    3.0202833810000014    Cr017
    2.9279999732999995    1.6904815729999996   21.1419836678999999    Cr018
    1.4639999867000000    2.5357223592000007    0.0000000000000000    Pd001
    1.4639999867000000    2.5357223592000007   18.1217002869000012    Pd002
    0.0000000000000000    0.0000000000000000    0.0000000000000000    Pd003
    0.0000000000000000    0.0000000000000000   18.1217002869000012    Pd004
    2.9279999734000000    5.0714447184000004    0.0000000000000000    Pd005
    2.9279999734000000    5.0714447184000004   18.1217002869000012    Pd006
    2.9279999737000000    3.3809631452000004    6.0405667619999983    Pd007
    2.9279999737000000    3.3809631452000004   24.1622670488999987    Pd008
    1.4639999871000002    5.9166855044000002    6.0405667619999983    Pd009
    1.4639999871000002    5.9166855044000002   24.1622670488999987    Pd010
    0.0000000004000002    3.3809631452000000    6.0405667619999983    Pd011
    0.0000000004000002    3.3809631452000000   24.1622670488999987    Pd012
    1.4639999867000000    4.2262039322000007   12.0811335249999985    Pd013
    1.4639999867000000    4.2262039322000007   30.2028338118999997    Pd014
    4.3919999599999997    4.2262039322000007   12.0811335249999985    Pd015
    4.3919999599999997    4.2262039322000007   30.2028338118999997    Pd016
    2.9279999732999995    1.6904815729999996   12.0811335249999985    Pd017
    2.9279999732999995    1.6904815729999996   30.2028338118999997    Pd018

3. 借助vaaspkit对pdcro2调整z方向使之和文献对应（z轴向下平移9.06085）

   vaspkit选项：403 -- 1 -- all -- 2 -- 0 0 -9.06085

   得到最终POSCAR

# MCIF绘制，输出vesta.mcif文件，该文件可以用vesta打开可视化

In [4]:
import argparse, numpy as np
from pymatgen.core import Structure

parser = argparse.ArgumentParser()
parser.add_argument("-m", "--magmom", default="MAGMOM.txt")
parser.add_argument("-p", "--poscar", default="POSCAR")
parser.add_argument("-o", "--mcif",   default="vesta.mcif")
args, _ = parser.parse_known_args()          # ← 让 notebook 忽略 -f …

# --- 读取 POSCAR ---
structure = Structure.from_file(args.poscar)

# --- 读取 MAGMOM ---
mag_flat = np.fromstring(open(args.magmom).read(), sep=" ")
magmoms  = mag_flat.reshape(-1, 3)

if len(structure) != len(magmoms):
    raise ValueError(f"Atom count mismatch: POSCAR {len(structure)}, MAGMOM {len(magmoms)}")

for i, site in enumerate(structure):
    site.properties["magmom"] = magmoms[i]

structure.to(fmt="mcif", filename=args.mcif)
print("MCIF written ➜", args.mcif)


MCIF written ➜ vesta.mcif
